# Text classification using LSTM

In [ ]:
import string
from collections import Counter
from tqdm import tqdm

import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchtext
import torchtext.datasets
from torchtext.vocab import build_vocab_from_iterator

from helpers import evaluate_one_epoch, plot_token_frequency_histogram, plot_token_count_histogram

이번 실습에서는 IMDB(Large Movie Review Dataset) 데이터셋을 활용하여 영화 리뷰의 감정을 분석해보겠습니다.

이 데이터셋은 총 5만개의 영화 리뷰로 구성되어 있으며, 각 리뷰에는 긍정(positive) 또는 부정(negative) 감정으로 레이블이 지정되어 있습니다.

먼저 IMDB 데이터셋을 간단히 살펴 보겠습니다.

In [ ]:
train_iter, test_iter = torchtext.datasets.IMDB(root = "/datasets/NLP")

for label, text in train_iter:
    print("----------------------------------------------------")
    print("Text :", text, "\n")
    print("Label:", label)
    break

`torchtext` 라이브러리를 사용 시 발생하는 deprecation warning는 모두 무시해도 괜찮습니다.

- Text: 영화 리뷰가 문자열 형태로 담겨있습니다.

- Label: 리뷰의 감정을 나타내는 정수값입니다
   - 1 : Negative reviews (부정적인 리뷰)
   - 2 : Positive reviews (긍정적인 리뷰)



## Custom Word-level tokenizer

자연어 처리(Natural Language Processing, NLP)를 위한 첫번째 과정은 텍스트 데이터(Corpus)를 일정한 단위인 <b>토큰(Token)</b>으로 분할하는 것입니다

### 토큰(Token) 이란?

토큰은 텍스트를 처리하기 위해 정의한 최소 단위를 의미합니다.

 - 사용자에 따라 단어(word), 글자(character), 또는 서브워드(subword) 등으로 정의할 수 있으며, 어떤 단위를 선택할지는 문제의 특성과 분석 목적에 따라 달라집니다.

텍스트를 토큰으로 나누는 과정을 <b>토큰화(Tokenization)</b>라고 부릅니다.

### 단어수준 토큰화(Word-level Tokenization)

대부분의 언어는 띄어쓰기를 이용해 문장을 단어 단위로 구분합니다. 

단어수준 토큰화(Word-level Tokenization)는 이처럼 텍스트 데이터를 의미 있는 기본 단위인 단어로 분리하는 작업입니다.

예를들어, `"good morning!"`은 `["good", "morning", "!"]`과 같이 토큰들의 `list`로 쪼개집니다.

여기서 `"!"`는 단어가 아니라 구두점(punctuation)이지만, 일반적으로 구두점은 별개의 토큰으로 처리됩니다. 따라서 토큰(token) 이라는 개념은 단어뿐 아니라 구두점, 숫자, 특수 기호까지 포괄하는 개념입니다.

토큰화가 완료되고 나면 문장은 <b>토큰들의 시퀀스(sequence of tokens)</b>로 표현됩니다

### 간단한 custom tokenzier

아래에 정의된 `simple_tokenizer`함수는 이해를 쉽게하기 위해 매우 단순화된 토큰화를 수행합니다.
 - 모든 대문자는 `.lower()`메서드를 이용하여 소문자로 변경합니다 .
 - `[".", "!", ";", "'", ...]` 등의 구두점(punctuation)은 모두 제거합니다.
 - 띄어쓰기를 기준으로 단어 단위 토큰화를 수행합니다.

In [ ]:
def simple_tokenizer(text):
    """ lowercase the text, remove punctuation, and split into words """
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = text.split()
    return tokens

In [ ]:
for label, text in train_iter:
    tokens = simple_tokenizer(text)
    print(f"Original Text:\n{text}")
    print(f"\nTokenzied Text:\n{tokens}")
    break

### Building a Vocabulary
모든 Corpus에 대하여 토큰화를 수행하고, 텍스트에 등장하는 <b>고유한(unique) 토큰들의 집합</b>을 얻을 수 있습니다. 

이 집합을 Vocabulary(vocab, $V$) 라고 부릅니다.


In [ ]:
print("Building vocabulary...")

token_counter = Counter()
for label, text in train_iter:
    tokens = simple_tokenizer(text)
    token_counter.update(tokens)

print(f"Total unique tokens: {len(token_counter)}")
print(f"Most common tokens: {token_counter.most_common(10)}")

대부분의 단어는 매우 적게 등장합니다. 예를 들어, 총 121,364개의 토큰들 중 약 100,000개는 10번 이하만 나타납니다.

In [ ]:
print(f"Number of tokens with frequency == 1: ", len([w for w in token_counter.values() if w <= 1]))
print(f"Number of tokens with frequency < 10: ", len([w for w in token_counter.values() if w < 10]))

plot_token_frequency_histogram(token_counter)

#### `<unk>` special token

자주 등장하지 않는 단어들은 모델 학습 과정에서 충분한 의미적 맥락을 제공하지 못하며, 모델 과적합(overfitting)의 원인이 될 수 있습니다.

따라서 일반적으로 <b>가장 자주 나타나는 상위 $N$개의 단어만을 유효한 vocabulary</b>로 선택하며, 그 외의 모든 단어는 특별 토큰 `<unk>` (unknown token) 으로 대체합니다.

이 접근법은 다음과 같은 장점을 가집니다.
- 모델이 훈련 중 `<unk>` 토큰을 학습하기 때문에, 처음 보는 단어가 나오더라도 주변 문맥을 기반으로 의미를 추론하도록 학습됨
- 훈련 데이터에는 없지만 검증/테스트 데이터에 새롭게 등장하는 단어가 있더라도, 모델이 어느정도 <b>일반화된 성능</b>을 발휘할 수 있음.

아래 코드는 상위 20,000개 토큰을 vocabulary로 사용하고, 그 외의 나머지 단어들을 표현하기 위한 `<unk>`토큰을 정의하는 코드입니다.


In [ ]:
most_common_tokens = token_counter.most_common(20000)

vocab = ['<unk>'] + [token for token, _ in most_common_tokens] # reserve index 0 for <unk>
token2idx = {token: idx for idx, token in enumerate(vocab)}
vocab_size = len(vocab)
UNK_IDX = token2idx['<unk>']

print(f"vocab_size = {vocab_size} (+1 for <unk>)")
print("vocab[:10]: ", vocab[:10])
print("token2idx dict: ", dict(list(token2idx.items())[:20]))

### One-hot encoding for text vectorization

토큰화가 완료된 후에는, 텍스트를 컴퓨터가 처리할 수 있도록 숫자 형태로 변환하는 과정이 필요합니다. 이를 <b>텍스트 벡터화(Text Vectorization)</b> 라고 합니다.

가장 기본적인 텍스트 벡터화 방법으로는 원-핫 인코딩(One-Hot Encoding)이 있습니다.

$$\mathbf{e}_i \in\{0,1\}^{|V|}$$

- One-hot Encoding된 벡터의 길이는 vocabulary의 크기 $|V|$와 동일합니다.
- 대부분의 원소가 `0`이므로 희소 벡터(sparse vector)입니다.
- 또한, 단어간의 의미적 유사성(예: like, love, adore)을 전혀 반영하지 못한다는 한계가 있습니다.


아래 코드는 IMDB 데이터셋을 토큰화한 후, One-hot Encoding을 적용하여 `Tensor`로 변환하는 과정입니다.

- 희귀 단어 (예: curiousyellow)는 모두 `<unk>` 토큰(`UNK_IDX = 0`)으로 대체됩니다.
- 최종적으로 `Tensor`는 `[# of tokens, vocab_size]`의 shape을 가집니다


In [ ]:
def one_hot_encode_tokens(token_indices, vocab_size):
    """
    Given a list of token indices, return a tensor of shape
    [len(token_indices), vocab_size] where each row is a one-hot vector.
    """
    onehot_tensor = torch.zeros(len(token_indices), vocab_size)
    for i, idx in enumerate(token_indices):
        onehot_tensor[i, idx] = 1.0
    return onehot_tensor

for label, text in train_iter:
    tokens = simple_tokenizer(text)
    token_idxs = [token2idx.get(t, UNK_IDX) for t in tokens]
    onehot_tensor = one_hot_encode_tokens(token_idxs, vocab_size)
    
    print(f"Text:\t {text}\n")
    print(f"Tokens:\t {tokens}\n")
    print("Indices:", token_idxs)
    print("\nOne-hot encoded Tensor shape:", onehot_tensor.shape)
    break

## TorchText tokenizer
PyTorch의 `TorchText`라이브러리는 텍스트 전처리를 간단히 하기 위한 tokenizer와 vocabulary를 자동으로 생성해주는 기능을 제공합니다

아래 코드는 TorchText의 기본 tokenizer인 `basic_english`를 사용하는 코드입니다.

In [ ]:
tokenizer = torchtext.data.utils.get_tokenizer("basic_english") 

for label, text in train_iter:
    tokens = tokenizer(text)
    print(f"Original Text:\n{text}")
    print(f"\nTokenzied Text:\n{tokens}")
    break

### Vocabularies

TorchText에서는 `build_vocab_from_iterator` 함수를 사용하여 `Vocab` (vocabulary) 객체를 생성할 수 있습니다.

 - 이 함수는 토큰 리스트(iterator)를 입력으로 받아 각 토큰의 빈도를 계산한 뒤, unique token들을 정수 인덱스와 매핑한 `Vocab` 객체를 반환합니다.
 - `Vocab` 객체는 토큰을 입력받아 해당 토큰의 인덱스를 반환하는 <b>사전(dictionary)</b> 역할을 합니다.
   - 예: `vocab["hello] → 1`,  `vocab["world"] → 2`, ...
 - `min_freq`: 지정된 최소 등장 빈도보다 적게 나타난 단어는 `<unk>`로 처리됩니다.
 - `specials`: 사전에 포함시킬 spatial token들의 리스트를 전달합니다.
   - 예: `<unk>` = unknown, `<pad>` = padding
   - 이러한 특수 토큰들은 vocabulary의 가장 앞 인덱스에 배치됩니다.
 - `set_default_index(vocab["<unk>"])` vocabulary에 존재하지 않는 토큰(OOV, out-of-vocabulary)이 입력될 경우 반환할 기본 인덱스를 `<unk>`토큰의 인덱스로 설정합니다.

---

<mark>주의</mark>
`Vocab`은 반드시 <b>학습(training) 데이터</b>만을 이용하여 생성해야 합니다.
 - 검증(validation) 또는 테스트(test) 데이터로 vocabulary를 만들면, 모델이 학습 중 알 수 없는 단어 정보를 미리 알게 되어 <b>정보 누출(information leakage)</b> 이 발생합니다.
 - 이는 validation/test 단계에서의 성능 평가를 낙관적(over-optimistic)으로 보이도록 만들어 부정확한 성능 평가로 이어질 수 있습니다.

In [ ]:
tokenizer = torchtext.data.utils.get_tokenizer("basic_english") 

token_lists = []
for label, text in train_iter:
    tokens = tokenizer(text)
    token_lists.append(tokens)

vocab = build_vocab_from_iterator(token_lists, specials=["<unk>", "<pad>"], min_freq = 2)
vocab.set_default_index(vocab["<unk>"])

`Vocab`객체는 사전처럼 동작합니다. 즉, token을 전달하면 그 token에 해당하는 정수 인덱스(index)가 반환됩니다.
- 각 토큰의 인덱스는 등장 빈도(frequency) 에 따라 할당됩니다.
- 즉, 가장 자주 등장하는 토큰일수록 작은 인덱스를 갖고, 드물게 등장하는 토큰일수록 큰 인덱스를 갖습니다.

In [ ]:
print("Index of token 'the` : ", vocab["the"])
print("Index of token 'hate`: ", vocab["hate"])

`get_itos()` 메서드
- itos는 int-to-string 의 약자로, `get_itos()`메서드는 인덱스를 토큰으로 변환하는 리스트를 반환합니다.

In [ ]:
vocab.get_itos()[:10]

`len` 메서드를 통해 `vocab_size`(number of unique tokens)를 얻을 수 있습니다

In [ ]:
vocab_size = len(vocab)
print(f"vocab_size = {vocab_size}")

Vocabulary에 없는 OOV 토큰을 전달하면 `<unk>` 토큰에 해당하는 인덱스가 리턴됩니다.

이는 앞서 `set_default_index` 메서드를 통해 기본값을 `vocab["<unk>"]`으로 설정했기 때문입니다.

In [ ]:
vocab["Jinhyun"]

아래 예시는 임의의 문장을 토큰화한 후, vocabulary를 사용해 각 토큰을 인덱스로 변환하는 과정입니다.

In [ ]:
sentence = "Hi, my name is Jinhyun! I'm enjoying this course."

print("Tokens\t\t:", tokenizer(sentence))
print("Token indices\t:", vocab(tokenizer(sentence)))
print("Index of <unk>:", vocab["<unk>"])

## 텍스트 전처리 파이프라인

지금까지 배운 과정들을 함수화하여 파이프라인으로 정리해 보겠습니다.
- `text_pipeline`: 입력 문장을 `tokenizer`로 토큰화한 뒤, 각 토큰을 `vocab`을 이용하여 정수 인덱스로 변환합니다.
- `label_pipeline`: `1`, `2`의 값을 가지는 IMDB 데이터셋의 라벨을 `0`, `1`의 값을 가지도록 변환합니다
  - `0`: Negative review
  - `1`: Positive review

In [ ]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

print(text_pipeline("Hello, world!"))
print(label_pipeline(1))

## Collate batch
딥러닝 모델은 일반적으로 <b>mini-batch</b>단위로 여러 개의 데이터를 동시에 처리합니다.

그러나 텍스트 데이터는 문장마다 포함된 토큰(token) 수가 제각각이기 때문에 이를 그대로 하나의 텐서(batch)로 묶기 어렵습니다.

아래 히스토그램을 통해 example별로 각 텍스트를 구성하는 토큰 수가 매우 다르다는 것을 확인할 수 있습니다.

In [ ]:
token_counts_per_text = [len(text_pipeline(text)) for label, text in train_iter]

plot_token_count_histogram(token_counts_per_text)

### Handles Sequences of Varying Length
RNN은 이론적으로 임의의 길이(sequence length)를 가진 입력 시퀀스를 처리할 수 있습니다. 

그러나 실제 학습에서는 `DataLoader`를 이용하여 입력 데이터들을 <b>미니배치(mini-batch)</b> 단위로 묶어서 처리합니다.

이때, 같은 mini-batch 안의 모든 입력 시퀀스들은 반드시 동일한 길이여야 `Tensor`로 변환될 수 있습니다.

### Padding

서로 다른 길이의 시퀀스들을 다루기 위해서 <b>padding</b>을 사용할 수 있습니다:
- 시퀀스 길이가 긴 경우 (Token length > `MAX_LEGNTH`) -> 뒷부분을 잘라냅니다(trucncate).
- 시퀀스 길이가 짧은 경우 (Token length < `MAX_LEGNTH`) -> `<pad>`토큰을 채워 길이를 `MAX_LEGNTH`로 맞춰줍니다.

예를 들어, `MAX_LEGNTH = 4`인 경우,
 - `["i", "really", "hate", "music", "videos"]` $\rightarrow$ `["i", "really", "hate", "music"]`
 - `["i", "love", "pizza"]` $\rightarrow$ `["i", "really", "hate", "<pad>"]`


`MAX_LEGNTH` 값을 설정하는 한가지 방법은 학습 데이터에서 가장 긴 문장의 길이를 선택하는 것입니다.
이번 실습에서는 학습 데이터에서 나타나는 토큰 개수 분포의 최빈값을 사용하여 `MAX_LEGNTH`값을 설정합니다

---

<mark>실습</mark> `collate_batch`함수를 완성하세요.

`DataLoader`는 입력 데이터들을 mini-batch로 묶어줄 때 `collate_fn` 함수를 호출합니다.

따라서, 이를 위한 함수를 직접 정의함으로써 입력데이터들의 길이를 맞춰줄 수 있습니다.

 - [`torch.cat`](https://docs.pytorch.org/docs/stable/generated/torch.cat.html) 함수를 활용하세요.
 - `nn.utils.rnn.pad_sequence`함수는 사용하지 <u>마세요</u>.

In [ ]:
MAX_LENGTH = 256

def collate_batch(batch):
    """
    Convert a batch of (label, text) pairs into fixed-length tensors.
    
    Each text is tokenized, then either truncated or padded with the "<pad>" token
    so that all texts in the batch have the same length (MAX_LENGTH).
    
    Args:
        batch: A list of tuples (label, text).
    
    Returns:
        text_tensor: Tensor of shape (batch_size, MAX_LENGTH) with token indices.
        label_tensor: Tensor of shape (batch_size,) with the corresponding labels.
        lengths_tensor: Tensor of shape (batch_size,) with each sequence’s true length.

    """
    label_list, text_list, length_list  = [], [], []
    pad_token_index = vocab["<pad>"]

    for _label, _text in batch:
        label_list.append(label_pipeline(_label))

        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.long)

        length_list.append(min(processed_text.shape[0], MAX_LENGTH))

        if processed_text.shape[0] > MAX_LENGTH:
            processed_text = ... # TODO
        else:
            padding = torch.full((MAX_LENGTH - processed_text.shape[0],), pad_token_index, dtype=torch.long)
            processed_text = ... # TODO

        text_list.append(processed_text)

    text_tensor = torch.stack(text_list)  # shape: (batch_size, MAX_LENGTH)
    label_tensor = torch.tensor(label_list, dtype=torch.long)
    lengths_tensor = torch.tensor(length_list, dtype=torch.long)
    return text_tensor, label_tensor, lengths_tensor

In [ ]:
batch = [
    (1, "It was totally a waste of time."),
    (2, "The movie was amazing!"),
]

text_tensor, label_tensor, length_tensor = collate_batch(batch)

print("text_tensor.shape:", text_tensor.shape)
print("Text tensor:\n", text_tensor)
print("\nLabel tensor:", label_tensor)
print("Length tensor:", length_tensor)

Expected output:
``` python
text_tensor.shape: torch.Size([2, 256])
Text tensor:
 tensor([[ 11,  17, 483,   6, 444,   7,  68,   3,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
 ...          
           1,   1,   1,   1],
        [  2,  21,  17, 476,  36,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
...
           1,   1,   1,   1]])

Label tensor: tensor([0, 1])
Length tensor: tensor([8, 5])

```

### Using `collate_batch` with `DataLoader`

정의한 `collate_batch`함수를 `DataLoader`에 전달하여 시퀀스 길이가 고정된 mini-batch를 얻을 수 있습니다.

In [ ]:
batch_size = 8

train_dataloader = DataLoader(train_iter, batch_size = batch_size, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_iter, batch_size = batch_size, shuffle=False, collate_fn=collate_batch)

for X, y, length in test_dataloader:
    print("Batch X shape:", X.shape)
    print("Batch y shape:", y.shape)

    print("First batch 3rd example:", X[2])
    print("First batch label:", y)
    print("First batch length:", length)
    break

## Word embedding

각 토큰을 $d$-차원의 실수 벡터(dense vector)로 매핑하여 표현하는 것을 <b>word embedding</b>이라고 합니다.

For each token $t_i \in V$ ($V$: vocabulary),
$$ t_i \mapsto \mathbf{e}_i \in \mathbb{R}^d , \quad(d \ll |V|)$$

 - one-hot벡터는 대부분의 값이 `0`이며 희소 벡터(sparse vector)인데 반해, 임베딩 벡터는 $d$차원의 dense vector로 단어의 의미를 더 효율적이고 컴팩트하게 표현합니다.

### Embedding Matrix

Vocabulary $V$내의 모든 token들에 대한 임베딩은 하나의 임베딩 행렬(Embedding Matrix) $E$로 표현할 수 있습니다:

$$ E \in \mathbb{R}^{|V| \times d} $$

- 토큰 인덱스 $t_i$에 대응하는 행(row)을 선택하면, 해당 토큰의 임베딩 벡터 $\mathbf{e}_i \in \mathbb{R}^d$를 얻을 수 있습니다.

### `nn.Embedding` 레이어 (embedding layer)

PyTorch에서는 `nn.Embedding`클래스를 통해 임베딩 레이어를 정의합니다. 이 레이어는 토큰의 인덱스를 입력받아 해당 토큰의 임베딩 벡터를 반환하는 <b>룩업 테이블(lookup table)</b>처럼 동작합니다.

``` python
nn.Embedding(num_embeddings, embedding_dim, padding_idx=None)
```

- `num_embeddings`: vocabulary 크기 $|V|$ (vocab_size)
- `embedding_dim`: embedding 벡터의 차원 $d$
- `padding_idx`: `<pad>` 토큰 인덱스를 지정하면, 해당 토큰의 임베딩 벡터는 항상 `0`으로 고정되어 학습되지 않습니다.
- `nn.Embedding`레이어는 내부적으로 $|V| \times d$ 크기의 Embedding Matrix를 갖고 있습니다.
  - token의 인덱스를 전달받으면 shape이 `[embed_dim]`인 임베딩 `Tensor`를 리턴해줍니다.
  - 만약 입력의 shape이 `[batch_size, seq_len]`이면, 출력의 shape은 `[batch_size, seq_len, embed_dim]`입니다

### word embdding을 얻는 방법
1. 랜덤 초기화 후 학습 (train from scractch)
   - Embedding Matrix를 랜덤으로 초기화하고, 모델 훈련 과정에서 <b>역전파(backpropagation)</b>를 통해 임베딩 벡터가 업데이트됩니다.
2. 사전학습 임베딩 활용 (pretrained word-embeddings)
   - Word2Vec, GloVe 등 사전 학습된 임베딩을 불러와 사용할 수 있습니다.

In [ ]:
embedding_layer = nn.Embedding(num_embeddings = 10, 
                               embedding_dim = 3, 
                               padding_idx = 0)
 
input_token_ids = torch.LongTensor([[1,2,4,5],[4,3,2,0]]) # (batch_size, seq_length) = (2, 4)
output_embedded = embedding_layer(input_token_ids)

print("nn.Embedding output shape:", output_embedded.shape)

## Text classification model

<mark>실습</mark> LSTM을 이용하여 IMBD 영화 리뷰 감정 분류를 수행하는 모델 `SentimentClassifier`를 완성하세요

1. `nn.Embedding`를 이용하여 토큰 인덱스를 $d$-차원 임베딩 벡터로 매핑합니다
2. 패딩된 시퀀스 처리 (`pack_padded_sequence`)
   - `nn.utils.rnn.pack_padded_sequence` ([docs](https://docs.pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pack_padded_sequence.html))함수를 이용하면 `<pad>`토큰을 무시하고 RNN/LSTM연산을 수행할 수 있습니다.
   - 이 함수는 padding된 입력 시퀀스를 압축하여 `PackedSequence` 객체를 리턴합니다
   - Arguments:
     - 시퀀스 길이 정보는 CPU 메모리에 있어야 합니다. `torch.Tensor.cpu`를 이용하세요.
     - `batch_first = True`로 설정하여 입력 텐서의 shape이 `(batch_size, seq_length, ...)` 임을 지정합니다
       - `batch_first = False`인 경우 입력 텐서의 shape은 `(seq_length, batch_size, ...)`입니다.
     - `enforce_sorted = False`로 설정하여, 입력 시퀀스들이 길이 순으로 정렬되어 있지 않음을 지정합니다

3. `nn.LSTM`에 `PackedSequence`를 전달하여 각 시점 $t$에서의 hidden state $\mathbf{h}_t$와 cell state $\mathbf{c}_t$를 계산합니다.
   - `nn.LSTM` 모듈의 출력값은 [docs](https://docs.pytorch.org/docs/stable/generated/torch.nn.LSTM.html)를 참고하세요.
4. 마지막 LSTM layer의 final hidden state $\mathbf{h}_T$를 이용하여 classification logits 을 계산합니다.
    $$\mathbf{\hat{y}} = \mathbf{W}_{hy}\mathbf{h}_T + \mathbf{b}_y$$
   - `self.fc`(`nn.Linear`)를 활용하세요

In [ ]:
class SentimentClassifier(nn.Module):
    """ Sentiment classification model using an LSTM. """

    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers, num_classes, pad_index):
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_index)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x, lengths):
        """
        Args:
            x: Tensor of shape (batch_size, seq_length) with token indices.
            lengths: Tensor of shape (batch_size,) with the true lengths of each sequence.

        Returns:
            logits: Tensor of shape (batch_size, num_classes)
        """

        ##### YOUR CODE START #####



        ##### YOUR CODE END #####
        
        return logits

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SentimentClassifier(
    vocab_size = len(vocab),
    embed_dim = 128,
    hidden_dim = 128,
    num_layers = 3,
    num_classes = 2,
    pad_index= vocab['<pad>']
).to(device)

X = torch.randint(0, 10, (8, 50)).to(device)  # (batch_size, seq_len)
lengths = torch.randint(1, 50, (8,)).to(device)  # lengths tensor with shape (batch_size, ) for each sequence in the batch
logits = model(X, lengths)

print("Logits shape:", logits.shape)  # (batch_size, num_classes)


## Gradient Clipping

RNN/LSTM 계열 모델은 훈련 과정에서 <b>gradient exploding</b> 문제가 발생할 수 있습니다. 이는 역전파 과정에서 기울기(gradient) 값이 지나치게 커져 학습이 불안정해지는 현상입니다.

이를 방지하기 위해 `torch.nn.utils.clip_grad_norm_` ([docs](https://docs.pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html)) 함수를 활용하여 gradient clipping을 수행합니다

In [ ]:
def train_one_epoch(model, device, dataloader, criterion, optimizer, epoch):
    """ train for one epoch """

    model.train()
    total_loss, total_correct, total_samples = 0.0, 0.0, 0

    for X, y, lengths in dataloader:
        X, y, lengths = X.to(device), y.to(device), lengths.to(device)

        logits = model(X, lengths)
        loss = criterion(logits, y)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()


        batch_size = y.shape[0]
        total_loss += loss.item() * batch_size
        total_correct += (logits.argmax(dim = 1) == y).type(torch.float).sum().item()
        total_samples += batch_size
        
    avg_loss = total_loss / total_samples
    accuracy = total_correct / total_samples
    return avg_loss, accuracy

In [ ]:
def train_sentiment_classifier():
    num_epochs = 5
    learning_rate = 1e-3       # Learning rate
    batch_size = 64

    vocab_size = len(vocab)
    embed_dim = 128
    hidden_dim = 128
    num_layers = 2
    num_classes = 2
    pad_index = vocab["<pad>"]
    

    device = "cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
    print(f"Using device: {device}")
    
    train_iter, test_iter = torchtext.datasets.IMDB(root = "/datasets/NLP")
    
    train_dataloader = DataLoader(train_iter, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
    test_dataloader = DataLoader(test_iter, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)
    
    model = SentimentClassifier(vocab_size = vocab_size, embed_dim = embed_dim, hidden_dim = hidden_dim,
                                num_layers = num_layers, num_classes = num_classes, pad_index = pad_index).to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print(f"Total number of model params : {sum(p.numel() for p in model.parameters() if p.requires_grad)}")
    
    for epoch in range(num_epochs):
        train_loss, train_accuracy = train_one_epoch(model, device, train_dataloader, criterion, optimizer, epoch)
        test_loss, test_accuracy = evaluate_one_epoch(model, device, test_dataloader, criterion, epoch)
        print(f"[Epoch {epoch + 1:02d}] Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")

    return model

In [ ]:
model = train_sentiment_classifier()

코드 구현이 잘 되었다면 별도의 하이퍼파라미터 튜닝(hyperparameter tuning)없이 `Test accuracy > 80%`를 달성하실 수 있습니다

## Prediction on new input text
이제 학습된 모델을 사용하여 새로운 영화 리뷰에 대한 감정을 예측해 보겠습니다.

아래 결과에서와 같이 학습된 LSTM모델은 단순히 단어의 존재 여부만 보는 것이 아니라
 - "not good"과 같이 단어의 순서와 조합을 고려하며,
 - "I thought the movie would be terrible, but it was actually amazing." 리뷰에서와 같이 문맥을 어느정도 이해하며 의미를 파악합니다.


In [ ]:
imdb_labels = {1: "👎", 2: "👍"}

def predict(model, text, text_pipeline):
    with torch.no_grad():
        X = torch.tensor(text_pipeline(text)).unsqueeze(0)
        length = torch.tensor([X.shape[1]])
        logits = model(X, length)
        probs = torch.softmax(logits, dim=1)
        predicted_label = logits.argmax(1).item() + 1
        return predicted_label, probs

sample_texts = [
    ("positive", "This movie was absolutely wonderful and touching."),
    ("negative", "The plot was boring and the acting was terrible."),
    ("positive", "I thought the movie would be terrible, but it was actually amazing."),
    ("negative", "The acting was good, but the plot ruined everything."),
    ("negative", "This movie is not good and not enjoyable"),
]

for sentiment, text in sample_texts:
    label, prob = predict(model.to("cpu"), text, text_pipeline)
    print(f"Input Text ({sentiment}): '{text}'")
    print(f"Predicted Label: {imdb_labels[label]}, Probabilities: {prob}\n")


# GloVe Word Embedding

In [ ]:
import torch
import torch.nn.functional as F
from torchtext.vocab import GloVe

## Pretrained Word Embedding

자연어처리(NLP)에서 학습 데이터셋이 작을 때, 사전학습된 word embedding을 사용하면 모델 성능을 크게 향상시킬 수 있습니다.

- 예를들어, 학습 데이터셋에 `"I love you."`라는 문장만 등장하고, 테스트 데이터에서 `"I adore you."`라는 문장이 등장했다고 가정해보겠습니다.
- `"adore"`라는 단어는 학습 데이터셋에서 전혀 등장하지 않았습니다.
- 하지만 사전학습된 word embedding에서는 `love`의 임베딩 벡터($\mathbf{e}_{\text{love}}$)와 `adore`의 임베딩 벡터($\mathbf{e}_{\text{adore}}$)가 매우 유사합니다.
- 따라서 모델은 `adore`라는 본 적 없는 새로운 단어라도 그 의미를 추론할 수 있습니다.
- 실제로 GloVe 임베딩에서는 `love`와 의미적으로 유사한 단어들(예:heart, dear, beloved, adore)이 임베딩 공간에서 서로 가까운 위치에 존재합니다.
- 이처럼 Word2Vec, GloVe와 같은 사전학습된 임베딩은 모델이 훈련 데이터에 없거나(OOV, out-of-vocabulary) 자주 등장하지 않는 단어들에 대해서도 일반화할 수 있도록 도와줍니다.

## GloVe 임베딩 
이번 실습에서는 사전학습된 GloVe (Global Vectors for Word Representation) 임베딩을 활용해보겠습니다.

In [ ]:
print("Loading pre-trained GloVe embeddings...")
glove = GloVe(name="6B", dim=50, cache="/datasets/NLP/glove")

`GloVe` 객체는 파이썬 사전(dictionary)처럼 동작하며, 단어(token)을 전달하면 word embedding을 반환받습니다.

In [ ]:
print("word embedding of 'cat': \n", glove["cat"])

## Cosine similarity
코사인 유사도(cosine similarity)는 두 벡터 간의 유사도를 측정할 때 자주 사용하는 방법입니다.

두 벡터 $u$와 $v$가 주어졌을때, cosine similarity는 다음과 같이 정의됩니다: 

$$\text{CosineSimilarity(u, v)} = \frac {u \cdot v} {||u||_2 ||v||_2} = cos(\theta) \tag{1}$$

* $u \cdot v$: 두 벡터간의 내적(dot product)
* $||u||_2$: 벡터의 L2 norm
* $\theta$: 두 벡터 사이의 각도
    * 만약 $u$와 $v$가 비슷하면, cosine similarity값은 1에 가까워집니다.
    * 만약 두 벡터의 차이가 크면, cosine similarity값은 0에 가까워 집니다.

In [ ]:
def compute_cosine_similarity(vec1, vec2):
    return F.cosine_similarity(vec1.unsqueeze(0), vec2.unsqueeze(0)).item()

word_a, word_b = "love", "like"
similarity = compute_cosine_similarity(glove[word_a], glove[word_b])
print(f"Cosine similarity between '{word_a}' and '{word_b}': {similarity:.4f}")

word_a, word_b = "love", "printer"
similarity = compute_cosine_similarity(glove[word_a], glove[word_b])
print(f"\nCosine similarity between '{word_a}' and '{word_b}': {similarity:.4f}")

## Top-k word retrieval

Word embedding 공간에서는 <b>의미적으로 비슷한 단어들이 서로 가까운 위치</b>에 존재합니다.
따라서 특정 단어를 기준으로, 임베딩 공간에서 가장 가까운 top-k개의 단어들을 검색하면 해당 단어와 의미적으로 유사한 단어들을 찾을 수 있습니다.


In [ ]:
def retrieve_top_k_similar(query, glove, k = 5):
    """ Returns the top-k most similar words to `query`, with scores. """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    assert query in glove.stoi, f"‘{query}’ not in GloVe vocabulary"

    query_vec = glove[query].to(device)    # [D]
    embedding_matrix = glove.vectors.to(device)  # [V, D]

    # Cosine similarity = dot / (||u|| * ||v||)
    cos_similarities = (embedding_matrix @ query_vec) / (embedding_matrix.norm(dim=1) * query_vec.norm() + 1e-8)  # [V]
    topk = torch.topk(cos_similarities, k + 1)  # +1 because query itself has sim = 1

    retrieved_words = []
    for idx, score in zip(topk.indices.tolist(), topk.values.tolist()):
        word = glove.itos[idx]
        if word != query:
            retrieved_words.append((word, score))
    return retrieved_words

query_word = "king"
top5 = retrieve_top_k_similar(query_word, glove, k=5)
print(f"Top 5 words similar to {query_word}:")
for w, sim in top5:
    print(f"    {w:10s}  cosine_similarity = {sim:.4f}")

## T-SNE visualization

고차원 word embedding을 2D로 투영해 의미적으로 유사한 단어들이 어떻게 가깝게 배치되는지를 시각화 해보겠습니다.

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def plot_tsne(words, categories, glove):
    vecs = torch.stack([glove[w] for w in words])

    tsne = TSNE(n_components=2, random_state = 42, perplexity = 30)
    proj = tsne.fit_transform(vecs.numpy())

    unique_category = sorted(set(categories))
    cmap = plt.get_cmap('tab10', len(unique_category))
    category_to_color = {cat: cmap(i) for i, cat in enumerate(unique_category)}

    plt.figure(figsize=(8, 6))
    for i, word in enumerate(words):
        cat = categories[i]
        plt.scatter(proj[i, 0], proj[i, 1],
                    color=category_to_color[cat])
        plt.annotate(word, (proj[i, 0], proj[i, 1]), fontsize=9)

    plt.title("t-SNE Projection of GloVe Embeddings")
    # plt.legend(loc="best", frameon=True)
    plt.tight_layout()
    plt.show()

words_example = [
    "cat", "dog", "lion", "tiger", "elephant", "whale", "shark", "eagle", "owl", "frog",
    "apple", "banana", "orange", "grape", "mango", "pineapple", "lemon", "cherry", "peach", "pear",
    "red", "blue", "green", "yellow", "purple", "orange", "pink", "brown", "black", "white",
    "car", "truck", "bicycle", "motorcycle", "airplane", "train", "boat", "ship", "submarine", "helicopter",
    "doctor", "engineer", "teacher", "artist", "lawyer", "chef", "nurse", "pilot", "scientist", "farmer",
    "happy", "sad", "angry", "surprised", "scared", "confident", "excited", "bored", "jealous", "proud",
    "computer", "smartphone", "internet", "software", "hardware", "robot", "drone", "server", "database", "algorithm",
    "circle", "square", "triangle", "rectangle", "hexagon", "octagon", "star", "heart", "diamond", "oval",
    "bread", "rice", "pasta", "pizza", "burger", "sushi", "salad", "soup", "steak", "taco",
    "run", "jump", "swim", "read", "write", "sing", "dance", "cook", "drive", "sleep",
]

words_categories = ["Animal"] * 10 + ["Fruit"] * 10 + ["Color"] * 10 + ["Vehicle"] * 10 + \
    ["Profession"] * 10 +  ["Emotion"] * 10 + ["Technology"] * 10 + ["Shape"] * 10 + ["Food"] * 10 + ["Verb"] * 10 

plot_tsne(words_example, words_categories, glove)

위 t-SNE 시각화 결과를 살펴보면 아래와 같은 흥미로는 결과들을 발견할 수 있습니다.
- 단어 그룹별로 자연스럽게 클러스터가 형성됩니다.
- orange는 색깔(color)과 과일(fruit)그룹의 중간에 위치하며, 동음이의어 현상을 잘 보여줍니다.
- apple은 smartphone과 가까이 있습니다.
- cook은 음식(food)과 관련된 단어들과 인접해 있으며, 요리 동사가 음식과 강하게 관련이 있음을 확인할 수 있습니다.

## Word analogy task

- Word analogy 문제는 다음과 같이 단어들 간의 관계를 유추하는 과제입니다:  
    "*a* is to *b* as *c* is to **____**". 

- 예를들어:  
    *man* : *woman* = *king* : *queen*.

### GloVe embedding and word analogy
- 사전학습된 GloVe 임베딩에서는 이러한 단어 간 관계가 벡터 공간 구조 속에서 자연스럽게 드러납니다.
- 즉, word embedding vector $\mathbf{e}_a, \mathbf{e}_b, \mathbf{e}_c, \mathbf{e}_d$들에 대해 다음과 같은 벡터 연산이 근사적으로 성립합니다. :   
    $$\mathbf{e}_b - \mathbf{e}_a + \mathbf{e}_c \approx \mathbf{e}_d $$

- 이 벡터 연산을 통해 analogy를 만족하는 새로운 단어 d를 찾을 수 있습니다.
- 이는 GloVe임베딩이 단순히 <b>개별 단어의 의미</b>뿐 아니라 <b>단어들 간의 관계</b>까지 학습했음을 보여줍니다.


In [ ]:
def word_analogy(word_a, word_b, word_c, glove, top_k = 5):
    """ 
    Solve word analogy tasks using vector arithmetic:
        word_b - word_a + word_c = ?
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    embedding_matrix = glove.vectors.to(device)  # [V, D]

    vec_a = glove[word_a].to(device)
    vec_b = glove[word_b].to(device)
    vec_c = glove[word_c].to(device)

    analogy_vector = vec_b - vec_a + vec_c

    cosine_similarities = (embedding_matrix @ analogy_vector) / (embedding_matrix.norm(dim=1) * analogy_vector.norm() + 1e-8)  # [V]

    # Get top-k most similar words
    top_values, top_indices = torch.topk(cosine_similarities, top_k)

    results = []
    for idx, score in zip(top_indices.tolist(), top_values.tolist()):
        candidate = glove.itos[idx]
        if candidate in {word_a, word_b, word_c}:
            continue
        results.append((candidate, score))

    return results

print("king - man + woman ≈ ?")
word_analogy("man", "king", "woman", glove, top_k=5)

# Seq2Seq Machine Translation

In [ ]:
import math
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import spacy

from tqdm import tqdm

from helpers import TranslationDataset

이번 실습에서는 프랑스어(French) 문장을 입력 받아 영어(English) 문장으로 번역하는 기계 번역 모델을 직접 구현하고 학습해 보겠습니다.

모델 구조는 [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215) 논문에서 제시한 Seq2Seq (Sequence-to-Sequence) 모델을 기반으로 합니다.

## Dataset
번역 모델 학습을 위한 `TranslationDataset`은 각 샘플을 튜플 `(source_sentence, target_sentence)`형식으로 반환합니다.
- source_sentence : 번역 대상이 되는 프랑스어 문장
- target_sentence : 번역 타겟이 되는 영어 문장

In [ ]:
dataset = TranslationDataset(path_tsv = "/datasets/NLP/eng-fra.txt")

test_size = int(0.1 * len(dataset))
valid_size = int(0.1 * len(dataset))
train_size = len(dataset) - test_size - valid_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size],
    generator=torch.Generator().manual_seed(42)
)
print(f"Train size: {len(train_dataset)}, Val size: {len(val_dataset)}, Test size: {len(test_dataset)}\n")

for source_sentence, target_sentence in train_dataset:
    print("Example source sentence:", source_sentence)
    print("Example target sentence:", target_sentence)
    break

## spaCy tokenizers
프랑스어와 영어 문장 토큰화를 위해서는 `spaCy` 라이브러리의 언어별 tokenizer를 활용합니다

- `fr_core_news_sm`: 프랑스어 토큰화 및 기본 언어 분석 기능을 제공하는 spaCy 모델
- `en_core_web_sm`: 영어 토큰화 및 기본 언어 분석 기능을 제공하는 spaCy 모델

In [ ]:
spacy_fr = spacy.load('fr_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')
tokenizer_french = get_tokenizer(lambda text: [tok.text.lower() for tok in spacy_fr.tokenizer(text)])
tokenizer_english = get_tokenizer(lambda text: [tok.text.lower() for tok in spacy_en.tokenizer(text)])

In [ ]:
text = "This is an example sentence."
tokenized_text = tokenizer_english(text)
print(tokenized_text)

## Vocabulary
각 언어(프랑스어, 영어)에 대해 정의된 tokenizer를 이용하여 `vocabulary`를 생성합니다.

특수 토큰(special tokens)으로는 다음 4가지를 사용합니다:
 - `<pad>` : 문장 길이를 맞추기 위한 padding 토큰
 - `<unk>` : 사전에 없는 단어 (OOV; out-of-vocabulary)
 - `<sos>` : 문장의 시작 (Start of Sentence)
 - `<eos>` : 문장의 끝 (End of Sentence)

In [ ]:
PAD_TOKEN = "<pad>"
UNK_TOKEN = "<unk>"
SOS_TOKEN = "<sos>"  # Start of Sentence
EOS_TOKEN = "<eos>"  # End of Sentence

SPECIAL_TOKENS = [PAD_TOKEN, UNK_TOKEN, SOS_TOKEN, EOS_TOKEN]

source_token_list = [tokenizer_french(source) for source, _ in train_dataset]
target_token_list = [tokenizer_english(target) for _, target in train_dataset]

source_vocab = build_vocab_from_iterator(source_token_list, specials=SPECIAL_TOKENS, min_freq = 2)
target_vocab = build_vocab_from_iterator(target_token_list, specials=SPECIAL_TOKENS, min_freq = 2)

source_vocab.set_default_index(source_vocab[UNK_TOKEN])
target_vocab.set_default_index(target_vocab[UNK_TOKEN])

print(f"Vocab sizes → French: {len(source_vocab)}, English: {len(target_vocab)}")

print("Source vocab (French): ", source_vocab.get_itos()[:10])
print("Target vocab (English): ", target_vocab.get_itos()[:10])

In [ ]:
PAD_TOKEN_IDX  = source_vocab[PAD_TOKEN]
SOS_TOKEN_IDX  = source_vocab[SOS_TOKEN]
EOS_TOKEN_IDX  = source_vocab[EOS_TOKEN]
assert PAD_TOKEN_IDX == target_vocab[PAD_TOKEN]
assert SOS_TOKEN_IDX == target_vocab[SOS_TOKEN]
assert EOS_TOKEN_IDX == target_vocab[EOS_TOKEN]

## collate_batch
<mark>실습</mark> `collate_seq2seq_batch`함수를 완성하세요
- `source_sentence`(프랑스어 문장)와 `target_sentence`(영어 문장) 각각에 대해 다음 과정들을 수행합니다:
  - <b>Tokenization</b> : 각 문장을 해당 언어의 tokenizer를 이용해 토큰 단위로 분할합니다.
    - `tokenizer_french`와 `tokenizer_english`를 이용하세요
  - <b>Vocabulary 매핑</b>: 각 토큰을 해당 언어의 `Vocabulary`를 이용하여 정수 인덱스(token indices)로 변환합니다
    - `source_vocab`과 `target_vocab`을 이용하세요
  - 문장의 시작에는 `<sos>`토큰을, 문장의 끝에는 `<eos>`토큰을 추가해줍니다.
    - `SOS_TOKEN`, `EOS_TOKEN` 또는 `SOS_TOKEN_IDX`, `EOS_TOKEN_IDX`를 이용하세요.
- <b>Padding</b>: [`torch.nn.utils.rnn.pad_sequence`](https://docs.pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html)함수를 이용하여 mini-batch내의 <b>가장 긴 시퀀스 길이</b>에 맞춰 padding을 수행합니다. 

In [ ]:
def collate_seq2seq_batch(batch):
    source_batch, target_batch = [], []
    for source_sentence, target_sentence in batch:
        source_indices = ...  # TODO
        target_indices = ...  # TODO
        source_batch.append(torch.tensor(source_indices, dtype=torch.long))
        target_batch.append(torch.tensor(target_indices, dtype=torch.long))

    # pad to max_length in batch
    source_batch = torch.nn.utils.rnn.pad_sequence(source_batch, padding_value=PAD_TOKEN_IDX, batch_first=True)
    target_batch = torch.nn.utils.rnn.pad_sequence(target_batch, padding_value=PAD_TOKEN_IDX, batch_first=True)
    return source_batch, target_batch

In [ ]:
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_seq2seq_batch)
valid_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_seq2seq_batch)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_seq2seq_batch)

for X, y in test_loader:
    print("Mini-batch X shape:", X.shape)
    print("Mini-batch y shape:", y.shape)
    print("\n1st source text:", X[0])
    print("1st target text:", y[0])
    break

print("\n<sos> token index:", SOS_TOKEN_IDX)
print("<eos> token index:", EOS_TOKEN_IDX)
print("<pad> token index:", PAD_TOKEN_IDX)

Expected output:
``` python
Mini-batch X shape: torch.Size([64, 27])
Mini-batch y shape: torch.Size([64, 23])

1st source text: tensor([  2,  19, 241, 136,  47,   9,   1,   4,   3,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])
1st target text: tensor([  2,   8,  62, 117,  13,   1,   7,  20,   4,   3,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0])

<sos> token index: 2
<eos> token index: 3
<pad> token index: 0
```

## Building the Seq2Seq Model

### Overview

![](resources/seq2seq1.png)

seq2seq 모델은 인코더(Encoder)와 디코더(Decoder) 두 개의 모듈로 구성됩니다. 
 - Encoder (인코더): 번역할 source 문장을 입력으로 받아 순환 신경망(RNN/LSTM/GRU)을 통해 문장 전체를 하나의 <b>context vector</b> $\mathbf{z}$로 요약합니다.
 - Decoder (디코더) : Encoder에서 생성된 context vector를 입력받아, target 언어 문장을 한 단어씩 순차적으로 생성하도록 학습됩니다.


### Encoder

입력 시퀀스 $X = (x_1, x_2, \dots, x_{T_x})$가 주어졌을때, Encoder LSTM은 각 time-step $t$에서 아래 수식에 따라 hidden state와 cell state를 업데이트합니다.

$$
(\mathbf{h}_t, \mathbf{c}_t) = \text{LSTM}_{\text{enc}}(x_t, \mathbf{h}_{t-1}, \mathbf{c}_{t-1})
$$

 - $(\mathbf{h}_t, \mathbf{c}_t)$는 지금까지 입력받은 토큰들 $x_{1:t}$에 대한 정보를 압축하여 담고 있는 상태 벡터입니다.
 - 최종적으로 문장 전체를 encoding하고 있는 context vector $\mathbf{z}$는 마지막 시점 $T_x$에서의 hidden state와 cell state로 정의됩니다

$$\mathbf{z} = (\mathbf{h}_{T_x}, \mathbf{c}_{T_x})$$

<mark>실습</mark> `Encoder` 모듈을 완성하세요
 - Embedding layer: `self.embedding`레이어를 이용하여 입력 토큰 인덱스를 임베딩 벡터로 변환합니다.
 - Droptout layer: 과적합 방지를 위해 `self.dropout`레이어에 임베딩 벡터를 통과시킵니다.
 - LSTM layer: `self.lstm`레이어를 통과시켜 context vector를 얻습니다.
   - LSTM 레이어의 출력값은 다음과 같이 세 가지로 구성됩니다 ([docs](https://docs.pytorch.org/docs/stable/generated/torch.nn.LSTM.html)):
     - `output` : 마지막 LSTM layer의 모든 time-step에서의 hidden state $(h_1, h_2, \dots, h_{T_x})$
     - `hidden` : 모든 LSTM layer의 final hidden state $h_{T_x}$.
     - `cell` : 모든 LSTM layer의 final cell state $h_{T_x}$.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers, pad_token_index, dropout = 0.5):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx = pad_token_index)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        """
        Args:
            src: Tensor of shape (batch_size, src_len) containing token indices for the source sentence.

        Returns:
            (hidden, cell): Tuple of tensors containing the final hidden and cell states of all LSTM layers.
        """
        
        ##### YOUR CODE START #####



        ##### YOUR CODE END #####

        return (hidden, cell)

In [ ]:
encoder = Encoder(vocab_size = len(source_vocab), embed_dim = 128, hidden_dim = 256,
                  num_layers = 2, pad_token_index = PAD_TOKEN_IDX, dropout = 0.5)

X = torch.randint(0, 100, (8, 20))  # (batch_size, src_len)
hidden, cell = encoder(X)

print("Encoder hidden state shape:", hidden.shape)  # (num_layers, batch_size, hidden_dim)
print("Encoder cell state shape:", cell.shape)      # (num_layers, batch_size, hidden_dim)

assert hidden.shape == (2, 8, 256), f"Expected hidden shape (2, 8, 256), but got {hidden.shape}"
assert cell.shape == (2, 8, 256), f"Expected cell shape (2, 8, 256), but got {cell.shape}"

### Decoder

Decoder는 Encoder로 부터 전달받은 context vector $\mathbf{z} = (\mathbf{h}_{T_x}, \mathbf{c}_{T_x})$를 초기 상태로 하여, 번역 대상이 되는 타겟 문장을 한 단어씩 순차적으로 생성합니다

Docoder LSTM의 초기 상태 $(\mathbf{h}_0^d, \mathbf{c}_0^d)$는 다음과 같이 Encoder LSTM의 마지막 state로 초기화됩니다.

$$(\mathbf{h}_0^d, \mathbf{c}_0^d) = \mathbf{z} = (\mathbf{h}_{T_x}, \mathbf{c}_{T_x})$$

이후 Docoder LSTM은 타겟 시퀀스 $Y = (y_1, y_2, \dots, y_{T_y})$의 각 time-step $t$에 대하여, 아래 수식에 따라 hidden state와 cell state를 업데이트합니다.

$$
(\mathbf{h}_t^d, \mathbf{c}_t^d) = \text{LSTM}_{\text{dec}}(y_t, \mathbf{h}_{t-1}^d, \mathbf{c}_{t-1}^d)
$$

또한 Decoder는 각 time-step $t$에서 다음 단어의 확률 분포(logits)를 예측합니다:
$$\mathbf{\hat{y}}_t = \mathbf{W}_{hy}\mathbf{h}_t^d + b_y, \quad t=1,2,...,T_y$$

<mark>실습</mark> `Decoder` 모듈을 완성하세요

 - `Decoder`는 한 time-step의 디코딩만 수행합니다. 즉 입력 토큰의 길이(sequence length)는 항상 `1`입니다. 
 - `seq_length = 1`이므로, 텐서의 차원 처리에 유의하세요.
   - 필요에 따라 [`unsqueeze`](https://docs.pytorch.org/docs/main/generated/torch.unsqueeze.html)와 [`squeeze`](https://docs.pytorch.org/docs/main/generated/torch.squeeze.html)메서드를 활용하여 텐서의 차원을 적절히 변환하세요.

계산 과정
 - Embedding layer: `self.embedding`레이어를 이용하여 입력 토큰 인덱스를 임베딩 벡터로 변환합니다.
 - Droptout layer: 과적합 방지를 위해 `self.dropout`레이어에 임베딩 벡터를 통과시킵니다.
 - LSTM layer: dropout을 통과한 임베딩 벡터와 previous hidden/cell states를 `self.lstm`레이어에 전달합니다.
   - LSTM layer의 출력값은 다음과 같습니다
     - `output` : 마지막 LSTM layer의 hidden state $h_t^d$
     - `hidden` : New hidden state  $\mathbf{h}_t^d$.
     - `cell` : New cell state $\mathbf{c}_t^d$.
 - fully-connected layer: LSTM의 출력 `output`를 `self.fc_out`레이어에 전달하여 다음 단어의 예측값(`logits`)을 계산합니다.

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers, pad_token_index, dropout = 0.5):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx = pad_token_index)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_token, hidden, cell):
        """
        Args:
            input_token: Tensor of shape (batch_size,) containing token index for the current time step.
            hidden: Previous hidden states of the decoder LSTM (or the encoder's final hidden state).
            cell: Previous cell states of the decoder LSTM (or the encoder's final cell state).

        Returns:
            logits: Tensor of shape (batch_size, vocab_size) containing prediction scores for the next token.
            (hidden, cell): Updated hidden and cell states.
        """
        ##### YOUR CODE START #####



        ##### YOUR CODE END #####
        
        return logits, (hidden, cell)

In [ ]:
dec = Decoder(vocab_size = len(target_vocab), embed_dim = 128, hidden_dim = 256,
              num_layers = 2, pad_token_index = PAD_TOKEN_IDX, dropout = 0.5)

input_token = torch.randint(0, 100, (8,))  # (batch_size,)
logits, (hidden, cell) = dec(input_token, hidden, cell)

print("Decoder logits shape:", logits.shape)        # (batch_size, vocab_size)
print("Decoder hidden state shape:", hidden.shape)  # (num_layers, batch_size, hidden_dim)
print("Decoder cell state shape:", cell.shape)      # (num_layers, batch_size, hidden_dim)

assert logits.shape == (8, len(target_vocab)), f"Expected logits shape (8, {len(target_vocab)}), but got {logits.shape}"
assert hidden.shape == (2, 8, 256), f"Expected hidden shape (2, 8, 256), but got {hidden.shape}"
assert cell.shape == (2, 8, 256), f"Expected cell shape (2, 8, 256), but got {cell.shape}"

### Seq2Seq

마지막으로 앞서 구현한 `Encoder`와 `Decoder`를 결합하여 `Seq2Seq` 모델을 완성합니다.

<mark>과제</mark> 아래 설명을 참고하여 `Seq2Seq`모델을 완성하세요

- `src`: 입력 시퀀스 $X = (x_1, x_2, \dots, x_{T_x})$
- `tgt`: 타겟 시퀀스 $Y = (y_1, y_2, \dots, y_{T_y})$

Forward 과정:
1. 예측값 $\hat{y}_t \ (t = 1, 2, \dots, T_y)$를 저장할 `output_logits`텐서를 초기화 합니다., 
2. <b>Encoding</b>: 입력 문장 `src`를 `encoder`에 전달하여 context vector (final hidden state & cell state)를 얻습니다.
3. For each iteration of the decoding:
   - `decoder`에 현재 입력 $y_t$와 이전 hidden state $\mathbf{h}_{t-1}^d$, 그리고 이전 cell states $\mathbf{c}_{t-1}^d$를 전달합니다.
      - 첫번째 iteration에서는 `encoder`의 final state $(\mathbf{h}_{T_x}, \mathbf{c}_{T_x})$가 전달됩니다.
   - `decoder`로부터 예측값 $\hat{y}_{t+1}$와 새로운 hidden/cell state $(\mathbf{h}_{t}^d, \mathbf{c}_{t}^d)$를 반환받습니다.
   - 예측값 $\hat{y}_{t+1}$를 `output_logits`텐서의 $t$ 위치에 저장합니다.
   - 난수를 발생시켜 "teacher force"을 적용할지 결정합니다.
      - "teacher force"을 적용할 경우, 다음 입력 토큰은 ground-truth $y_{t+1}$입니다.
      - "teacher force"을 적용하지 않을 경우, 다음 입력 토큰은 `decoder`의 예측값 $\arg\max(\hat{y}_{t+1})$입니다.

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.target_vocab_size = self.decoder.fc_out.out_features
        
    def forward(self, src, tgt, teacher_forcing_prob = 0.5):
        """
        Args:
            src: Tensor of shape (batch_size, source_seq_len) containing token indices for the source sentences.
            tgt: Tensor of shape (batch_size, target_seq_len) containing token indices for the target sentences.
            teacher_forcing_prob: Probability of using the ground-truth token as the next input.

        Returns:
            output_logits: Tensor of shape (batch_size, target_seq_len, target_vocab_size)
        """
        batch_size, target_seq_len = tgt.shape
        output_logits = torch.zeros(batch_size, target_seq_len, self.target_vocab_size).to(tgt.device)

        # Encode source sequence
        (hidden, cell) = ...  # TODO

        # First decoder input is <sos>
        current_input_token = tgt[:, 0]  # (batch_size,)

        # Decode one step at at time
        for t in range(1, target_seq_len):
            decoder_logits, (hidden, cell) = ...  # TODO
            output_logits[:, t, :] = ...  # TODO

            # choose next input token: teacher forcing or model’s own prediction
            use_teacher_forcing = torch.rand(1).item() < teacher_forcing_prob
            predicted_token  = decoder_logits.argmax(dim = 1)  # (batch_size,)
            current_input_token = tgt[:, t] if use_teacher_forcing else predicted_token

        return output_logits

In [ ]:
embed_dim = 256
hidden_dim = 512
num_layers = 2

enc = Encoder(vocab_size = len(source_vocab), embed_dim = embed_dim, hidden_dim = hidden_dim,
              num_layers = num_layers, pad_token_index = PAD_TOKEN_IDX, dropout = 0.5)

dec = Decoder(vocab_size = len(target_vocab), embed_dim = embed_dim, hidden_dim = hidden_dim,
              num_layers = num_layers, pad_token_index = PAD_TOKEN_IDX, dropout = 0.5)

model = Seq2Seq(enc, dec)


X = torch.randint(0, 100, (32, 20)) # (batch_size, src_len)
y = torch.randint(0, 100, (32, 30)) # (batch_size, tgt_len)

logits = model(X, y, teacher_forcing_prob=0.5)
print("Seq2Seq logits shape:", logits.shape)  # (batch_size, tgt_len, target_vocab_size)

assert logits.shape == (32, 30, len(target_vocab)), f"Expected logits shape (32, 30, {len(target_vocab)}), but got {logits.shape}"

### Training Loop

학습 루프는 전형적인 딥러닝 학습 과정과 거의 동일하나, 다음과 같은 차이가 있습니다:

- Loss 계산에서 `<sos>` 제외: `<sos>` 토큰에는 대응되는 예측값이 없으므로, Loss를 계산하기 전 `tgt` 텐서와 `output` 텐서의 첫번째 컬럼을 잘라냅니다
- loss계산을 위해 `output` 텐서의 shape을 `(batch_size, tgt_len, vocab_size)`에서 `(batch_size * tgt_len, vocab_size)`으로 reshape합니다.
- `clip_grad_norm_`함수를 통해 gradient clipping을 수행하여 gradient explosion을 방지합니다.


In [ ]:
def train_one_epoch(model, device, dataloader, criterion, optimizer, epoch, clip = 1.0):
    model.train()

    total_loss = 0.0

    for src, tgt in tqdm(dataloader, desc=f'Training Epoch {epoch + 1}', total = len(dataloader), leave=False):
        src, tgt = src.to(device), tgt.to(device)

        output = model(src, tgt) # (batch_size, tgt_len, vocab_size)

        # exclude first token (<sos>) from loss, flatten
        output_dim = output.shape[-1]
        output_flat = output[:, 1:].reshape(-1, output_dim)
        tgt_flat = tgt[:, 1:].reshape(-1)

        loss = criterion(output_flat, tgt_flat)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

### Evaluation Loop
 - 평가 단계에서는 teacher forcing을 적용하지 않습니다. 
 - 즉, 모델은 정답 토큰을 참조하지 않고, 오직 <b>자신이 생성한 이전 예측값을 다음 입력으로 사용하여</b> 문장을 생성합니다. 이를 <b>Autoregressive generation</b>이라고 합니다.

In [ ]:
def evaluate_one_epoch(model, device, dataloader, criterion, epoch = 0):
    model.eval()

    total_loss = 0

    with torch.no_grad():
        for src, tgt in tqdm(dataloader, desc='Validation/Test', total = len(dataloader), leave=False):
            src, tgt = src.to(device), tgt.to(device)
            output = model(src, tgt, teacher_forcing_prob=0.0)

            output_dim = output.shape[-1]
            output_flat = output[:, 1:].reshape(-1, output_dim)
            tgt_flat = tgt[:, 1:].reshape(-1)

            loss = criterion(output_flat, tgt_flat)
            
            total_loss += loss.item()

    return total_loss / len(dataloader)

<mark>실습</mark> 직접 Seq2Seq 번역 모델을 학습해보세요.
 - 학습에는 <u>약 20분</u> 정도 소요됩니다.
 - Loss function으로는 `CrossEntropyLoss`를 사용하며, `ignore_index = PAD_TOKEN_IDX`으로 세팅해줍니다.
   - 이를 통해, loss 계산 시 `<pad>` 토큰에 대응되는 예측값들은 모두 무시되어 학습에 영향을 주지 않습니다.

In [ ]:
def train_seq2seq():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    SOURCE_VOCAB_SIZE  = len(source_vocab)
    TARGET_VOCAB_SIZE  = len(target_vocab)
    embed_dim       = 256
    hidden_dim      = 512
    num_layers      = 2
    encoder_dropout = 0.2
    decoder_dropout = 0.2
    grad_clip       = 1.0

    num_epochs    = 7
    batch_size = 64


    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, 
                              shuffle=True, num_workers= 4, collate_fn=collate_seq2seq_batch)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, 
                              shuffle=False, num_workers= 4, collate_fn=collate_seq2seq_batch)
    test_dataloader  = DataLoader(test_dataset, batch_size=batch_size, 
                              shuffle=False, num_workers= 4, collate_fn=collate_seq2seq_batch)

    enc = Encoder(vocab_size = SOURCE_VOCAB_SIZE, embed_dim = embed_dim, hidden_dim = hidden_dim,
                num_layers = num_layers, pad_token_index = PAD_TOKEN_IDX, dropout = encoder_dropout)
    dec = Decoder(vocab_size = TARGET_VOCAB_SIZE, embed_dim = embed_dim, hidden_dim = hidden_dim,
                num_layers = num_layers, pad_token_index = PAD_TOKEN_IDX, dropout = decoder_dropout)
    model = Seq2Seq(enc, dec).to(device)

    print(f'The model has {sum(p.numel() for p in model.parameters() if p.requires_grad)} trainable parameters')

    criterion = nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_IDX)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(num_epochs):
        train_loss = train_one_epoch(model, device, train_dataloader, criterion, optimizer, epoch, grad_clip)
        val_loss = evaluate_one_epoch(model, device, val_dataloader, criterion, epoch)
        print(f"Epoch {epoch + 1:02} | Train Loss: {train_loss:.3f} | Val. Loss: {val_loss:.3f}")
        print(f'\t | Train PPL: {math.exp(train_loss):7.3f} | Val. PPL: {math.exp(val_loss):7.3f}')

    test_loss = evaluate_one_epoch(model, device, test_dataloader, criterion)
    print(f"| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |")

    return model

In [ ]:
model = train_seq2seq()

## Autoregressive sentence translation

이제 학습된 Seq2Seq 모델을 사용하여 새로운 문장을 번역해보겠습니다.

번역 과정은 다음과 같습니다:
1. 번역할 입력 문장을 `encoder`에 전달하여 context vector `(hidden, cell)`를 얻습니다.
2. `decoder`의 첫번째 입력 token을 `<sos>`으로 설정하고, 이를 기반으로 다음 토큰을 예측합니다.
3. 예측된 토큰을 다음 입력으로 사용하여, `<eos>` 토큰을 만날때까지 반복적으로 next token prediction을 수행합니다.

In [ ]:
def translate_sentence(sentence, model, max_len = 50):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.eval()
    tokens = [SOS_TOKEN] + tokenizer_french(sentence) + [EOS_TOKEN]
    source_indices = torch.tensor(source_vocab(tokens), dtype=torch.long).unsqueeze(0).to(device)

    with torch.no_grad():
        (hidden, cell) = model.encoder(source_indices)

        input_tok = torch.tensor([SOS_TOKEN_IDX], device=device)
        result_tokens = []
        for _ in range(max_len):
            pred, (hidden, cell) = model.decoder(input_tok, hidden, cell)
            next_token_idx = pred.argmax(1).item()
            if next_token_idx == EOS_TOKEN_IDX:
                break
            result_tokens.append(target_vocab.lookup_token(next_token_idx))
            input_tok = torch.tensor([next_token_idx], device=device)

    return result_tokens

example_sentences = [
    "Je t'aime.",
    "Je me sens heureux"
]

for example_fr in example_sentences:
    print(f"FR ➡️ EN  : '{example_fr}'")
    print(f"Predicted: {translate_sentence(example_fr, model)}\n")

# Using pretrained Word Embeddings

<mark>optional 실습</mark> 사전 학습된 GloVe 임베딩(Pretrained Embedding) 활용하여 `SentimentClassifier` 모델을 학습해보고, 기존 학습 방식과 성능을 비교해보세요

- 본 실습은 선택사항입니다. 
- <mark>주의</mark> 추가 실습을 시작하기 전에, 지금까지 완료한 과제 코드를 `git push`하여 과제 채점에 영향을 주지 않도록 하세요.

In [ ]:
import torch
import torch.nn as nn
from torchtext.vocab import GloVe

In [ ]:
def build_embedding_layer_from_glove(glove, vocabulary):
    """
    Create a PyTorch nn.Embedding layer with weights initialized from pre-trained GloVe embeddings.

    Args:
        glove: Pre-trained GloVe embeddings.
        vocabulary: List of words in your vocabulary.

    Returns:
        A PyTorch nn.Embedding layer with pre-loaded weights.
    """
    embedding_dim = glove.dim
    vocab_size = len(vocabulary)
    weights = torch.zeros(vocab_size, embedding_dim)

    for idx, word in enumerate(vocabulary):
        weights[idx] = glove[word]

    # Setting freeze=True prevents the weights from being updated during training.
    embedding_layer = nn.Embedding.from_pretrained(weights, freeze=True)
    return embedding_layer

glove = GloVe(name="6B", dim=50, cache="/datasets/NLP/glove")

custom_vocabulary = glove.stoi.keys() # replace with your own vocabulary
embedding_layer = build_embedding_layer_from_glove(glove, custom_vocabulary)

print("vocabulary size:", len(custom_vocabulary))
print("Embedding layer weights.shape : ", embedding_layer.weight.shape)